Trying to update the Webscraping Indeed Notebook to Python 3

In [10]:
%load_ext autoreload
%autoreload 2

In [11]:
# API Calls
import requests
# Parse HTML
import bs4
# Handle Dataframes (excel data)
import pandas as pd
# Plotting library
# import matplotlib as plt

## Fetching and Cleaning Indeed Search Data

In [12]:
from indeed_scraper import search_indeed, clean_data, save_data, posting_scraper

In [13]:
# Fetch Page Information for Indeed Search
query = "machine+learning"
cities = ["New+York%2C+NY"]
max_results_per_city = 100
null_value = "NA"

df = search_indeed(query, cities, max_results_per_city, null_value)
df = clean_data(df)
filename = save_data(df, query)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\pandas\core\generic.py:4619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
E:\Users\Jonathan\Coding\Python\career-posting-scraper\indeed_scraper.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_copy['url'] = "h" + "ttps://www.indeed.com/viewjob?jk=" + df_copy['url']


## Getting Data Per Posting Page (WIP)

In [35]:
# # Getting one posting worth of data
# url = data.loc[:, 'url'].values[0]
# html = requests.get(url).text
# soups = bs4.BeautifulSoup(html, "html.parser")
# # Print out job description as one srting
# main_content = soups.find('div', {'class': "jobsearch-JobComponent icl-u-xs-mt--sm jobsearch-JobComponent-bottomDivider"})
# job_description = soups.find('div', {'class': "jobsearch-JobComponent-description icl-u-xs-mt--md"})
# # job_description.get_text("  ", strip=True).strip()

In [34]:
# # Print out job description as one srting
# main_content = soups.find('div', {'class': "jobsearch-JobComponent icl-u-xs-mt--sm jobsearch-JobComponent-bottomDivider"})
# job_description = soups.find('div', {'class': "jobsearch-JobComponent-description icl-u-xs-mt--md"})
# # job_description.get_text("  ", strip=True).strip()

In [14]:
## Getting Data Per Posting Page (WIP)
data = pd.read_csv(f"{filename}.csv", index_col=0)
desc_dataframe = posting_scraper(data, filename)
data['desc'] = desc_dataframe['desc']

data.to_csv(f'{filename}.csv', sep=',', encoding='utf-8')

## Count Vectorizer on File
Seeing most popular words in job posting.

In [15]:
# filename = "machine+learning-2018_11_24-185749"
job_info = pd.read_csv(f'{filename}.csv', index_col=0)

In [16]:
descriptions = []
for txtfile in job_info['desc']:
    with open(txtfile, 'r', encoding='utf-8') as the_file:
        descriptions.append(the_file.read().lower())

In [183]:
# https://medium.com/@cristhianboujon/how-to-list-the-most-common-words-from-text-corpus-using-scikit-learn-dad4d0cab41d
def get_top_n_words(corpus, stop_words=None):
    """
    List the top n words in a vocabulary according to occurrence in a text corpus.
    
    get_top_n_words(["I love Python", "Python is a language programming", "Hello world", "I love the world"]) -> 
    [('python', 2),
     ('world', 2),
     ('love', 2),
     ('hello', 1),
     ('is', 1),
     ('programming', 1),
     ('the', 1),
     ('language', 1)]
    """
    vectorizer = CountVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names()
    sum_words = X.sum(axis=0).tolist()[0]
    words_freq = zip(feature_names, sum_words)
    words_freq =sorted(words_freq, key = lambda x: -x[1])
    return words_freq

In [31]:
%autoreload 2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from more_stop_words import more_stop_words
custom_stop_words = ENGLISH_STOP_WORDS.union(more_stop_words)
# from sklearn.model_selection import train_test_split

In [32]:
# print(descriptions)
vectorizer = CountVectorizer(stop_words=custom_stop_words)
X = vectorizer.fit_transform(descriptions)
feature_names = vectorizer.get_feature_names()
sum_words = X.sum(axis=0).tolist()[0]
words_freq = zip(feature_names, sum_words)
words_freq =sorted(words_freq, key = lambda x: -x[1])

In [33]:
final_map = [word for word in words_freq if word[1] > 10]
# print(words_freq[:])
final_map

[('research', 175),
 ('models', 111),
 ('ml', 97),
 ('ai', 91),
 ('deep', 91),
 ('python', 90),
 ('language', 79),
 ('analytics', 72),
 ('java', 70),
 ('york', 69),
 ('natural', 55),
 ('financial', 49),
 ('applied', 45),
 ('statistics', 43),
 ('quantitative', 42),
 ('scientists', 42),
 ('modeling', 41),
 ('nlp', 38),
 ('scientist', 38),
 ('artificial', 36),
 ('methods', 35),
 ('sales', 33),
 ('tensorflow', 33),
 ('aws', 31),
 ('phd', 31),
 ('optimization', 30),
 ('training', 30),
 ('amazon', 29),
 ('conferences', 29),
 ('hadoop', 29),
 ('statistical', 29),
 ('distributed', 28),
 ('vision', 28),
 ('analytical', 27),
 ('spark', 27),
 ('education', 26),
 ('frameworks', 26),
 ('model', 26),
 ('scala', 26),
 ('deploying', 25),
 ('master', 25),
 ('startup', 25),
 ('feature', 24),
 ('group', 24),
 ('mining', 24),
 ('senior', 24),
 ('speech', 24),
 ('state', 24),
 ('advanced', 23),
 ('agile', 23),
 ('ideas', 23),
 ('multiple', 23),
 ('proven', 23),
 ('scalable', 23),
 ('text', 23),
 ('marketin